# Градиентные оптимизаторы

В ноутбуке рассматриваются три метода:
1. Momentum
2. Nesterov Momentum
3. RMSProp

Для каждого метода показываются:
- итоговая найденная точка
- динамическая траектория движения к минимуму


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

plt.rcParams['figure.figsize'] = (10, 5)
plt.rcParams['axes.grid'] = True


## Вспомогательные функции визуализации


In [ ]:
def animate_trajectory(func, x_history, x_min=-6, x_max=20, title='Движение к минимуму'):
    x_grid = np.linspace(x_min, x_max, 500)
    y_grid = func(x_grid)

    fig, ax = plt.subplots()
    ax.plot(x_grid, y_grid, color='steelblue', linewidth=2, label='f(x)')

    point, = ax.plot([], [], 'ro', markersize=7, label='Текущая точка')
    path_line, = ax.plot([], [], color='tomato', linewidth=1.5, alpha=0.8, label='Траектория')

    ax.set_xlim(x_min, x_max)
    y_pad = 0.15 * (np.max(y_grid) - np.min(y_grid) + 1e-9)
    ax.set_ylim(np.min(y_grid) - y_pad, np.max(y_grid) + y_pad)
    ax.set_title(title)
    ax.set_xlabel('x')
    ax.set_ylabel('f(x)')
    ax.legend(loc='best')

    xs = []
    ys = []

    def init():
        point.set_data([], [])
        path_line.set_data([], [])
        return point, path_line

    def update(frame):
        x = x_history[frame]
        y = func(x)
        xs.append(x)
        ys.append(y)

        point.set_data([x], [y])
        path_line.set_data(xs, ys)
        return point, path_line

    ani = FuncAnimation(
        fig,
        update,
        frames=len(x_history),
        init_func=init,
        interval=80,
        blit=True,
        repeat=False,
    )
    plt.close(fig)
    return HTML(ani.to_jshtml())


def plot_final_result(func, x_history, x_min=-6, x_max=20, title='Результат оптимизации'):
    x_grid = np.linspace(x_min, x_max, 500)
    y_grid = func(x_grid)

    x_final = x_history[-1]
    y_final = func(x_final)

    plt.figure()
    plt.plot(x_grid, y_grid, color='steelblue', linewidth=2, label='f(x)')
    plt.scatter([x_final], [y_final], color='crimson', s=80, label=f'Найдено: x={x_final:.4f}')
    plt.title(title)
    plt.xlabel('x')
    plt.ylabel('f(x)')
    plt.legend()
    plt.show()


## Задание 1. Momentum (метод импульса)


In [ ]:
def f_momentum(x):
    return -0.5 * x + 0.2 * x**2 - 0.01 * x**3 - 0.3 * np.sin(4 * x)


def df_momentum(x):
    return -0.5 + 0.4 * x - 0.03 * x**2 - 1.2 * np.cos(4 * x)


In [ ]:
eta = 0.1
gamma = 0.8
n_steps = 200
x0 = -3.5

x = x0
v = 0.0
x_hist_momentum = [x]

for _ in range(n_steps):
    v = gamma * v + (1 - gamma) * eta * df_momentum(x)
    x = x - v
    x_hist_momentum.append(x)

print(f'Momentum: x* = {x_hist_momentum[-1]:.6f}, f(x*) = {f_momentum(x_hist_momentum[-1]):.6f}')


In [ ]:
plot_final_result(
    f_momentum,
    x_hist_momentum,
    x_min=-6,
    x_max=12,
    title='Momentum: итоговая точка',
)


In [ ]:
animate_trajectory(
    f_momentum,
    x_hist_momentum,
    x_min=-6,
    x_max=12,
    title='Momentum: движение к минимуму',
)


## Задание 2. Nesterov Momentum


In [ ]:
def f_nesterov(x):
    return 0.4 * x + 0.1 * np.sin(2 * x) + 0.2 * np.cos(3 * x)


def df_nesterov(x):
    return 0.4 + 0.2 * np.cos(2 * x) - 0.6 * np.sin(3 * x)


In [ ]:
eta = 0.1
gamma = 0.7
n_steps = 500
x0 = 4.0

x = x0
v = 0.0
x_hist_nesterov = [x]

for _ in range(n_steps):
    v = gamma * v + (1 - gamma) * eta * df_nesterov(x - gamma * v)
    x = x - v
    x_hist_nesterov.append(x)

print(f'Nesterov: x* = {x_hist_nesterov[-1]:.6f}, f(x*) = {f_nesterov(x_hist_nesterov[-1]):.6f}')


In [ ]:
plot_final_result(
    f_nesterov,
    x_hist_nesterov,
    x_min=-6,
    x_max=12,
    title='Nesterov: итоговая точка',
)


In [ ]:
animate_trajectory(
    f_nesterov,
    x_hist_nesterov,
    x_min=-6,
    x_max=12,
    title='Nesterov: движение к минимуму',
)


## Задание 3. RMSProp


In [ ]:
def f_rmsprop(x):
    return 2 * x + 0.1 * x**3 + 2 * np.cos(3 * x)


def df_rmsprop(x):
    return 2 + 0.3 * x**2 - 6 * np.sin(3 * x)


In [ ]:
eta = 0.5
alpha = 0.8
eps = 0.01
n_steps = 200
x0 = 4.0

x = x0
G = 0.0
x_hist_rmsprop = [x]

for _ in range(n_steps):
    grad = df_rmsprop(x)
    G = alpha * G + (1 - alpha) * grad**2
    step = eta * grad / (np.sqrt(G) + eps)
    x = x - step
    x_hist_rmsprop.append(x)

print(f'RMSProp: x* = {x_hist_rmsprop[-1]:.6f}, f(x*) = {f_rmsprop(x_hist_rmsprop[-1]):.6f}')


In [ ]:
plot_final_result(
    f_rmsprop,
    x_hist_rmsprop,
    x_min=-6,
    x_max=12,
    title='RMSProp: итоговая точка',
)


In [ ]:
animate_trajectory(
    f_rmsprop,
    x_hist_rmsprop,
    x_min=-6,
    x_max=12,
    title='RMSProp: движение к минимуму',
)


## Что можно менять

- `eta`: скорость обучения
- `gamma`: коэффициент импульса
- `alpha`: коэффициент накопления квадрата градиента
- `n_steps`: число шагов
- `x0`: начальная точка

Меняй параметры и сравнивай траектории: так хорошо видно скорость сходимости и колебания.
